## Setup

Step 1: Download the two models from this link and store them in `./checkpoints`.

Step 2: Use this notebook to load the models.

Step 3: If needed, convert the models to a HuggingFace version. (this is TransformerLens and might have different variable names, but the model is functionally identical to gpt2-small.)

Step 4: Interp.

In [1]:
from transformer_lens import HookedTransformer
import torch
from plotly import graph_objects as go
import plotly.express as px
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"

/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = './checkpoints/'

In [5]:
# clustered model

clustered_model = HookedTransformer.from_pretrained("gpt2-small")
clustered_model.to(device)
# path_clustered = '/home/b-sgolechha/research/nn-modularity/language-models/checkpoints/wiki_modular_mlp_in_model_epoch_6.pt'
path_clustered = "/Users/maheepchaudhary/pytorch/Projects/nn-modularity/interp-gains-gpt2small/data/models/new_wiki_modular_mlp_in_model_epoch_1.pt"
clustered_model.load_state_dict(torch.load(path_clustered, map_location=device))

/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cpu


/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_17436/3633571846.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clustered_model.load_state_dict(torch.load(pa

<All keys matched successfully>

In [4]:
# unclustered model

unclustered_model = HookedTransformer.from_pretrained("gpt2-small")
unclustered_model.to(device)
# path_unclustered = '/home/b-sgolechha/research/nn-modularity/language-models/checkpoints/wiki_non_modular_mlp_in_model_epoch_2.pt'
path_unclustered = "/Users/maheepchaudhary/pytorch/Projects/nn-modularity/interp-gains-gpt2small/data/models/new_wiki_non_modular_mlp_in_model_epoch_1.pt"
unclustered_model.load_state_dict(torch.load(path_unclustered, map_location=device))

/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cpu


/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_17436/866031515.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unclustered_model.load_state_dict(torch.load(p

<All keys matched successfully>

In [6]:
# sanity check to see if the models are loaded correctly

input = 'The color of the darkness is'

clustered_output = clustered_model.generate(input, max_new_tokens=20)
unclustered_output = unclustered_model.generate(input, max_new_tokens=20)

print('Clustered model output: ', clustered_output)

print('Unclustered model output: ', unclustered_output)

  0%|          | 0/20 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 20/20 [00:00<00:00, 29.66it/s]

Clustered model output:  The color of the darkness is <unk> and follows the east @-@ South direction of the country at the barrow,
Unclustered model output:  The color of the darkness is up to 300 % compared to the other environments, although not always four @-@ shades considerably darker


In [7]:
# get loss and accuracy on wiki dataset

from transformer_lens.evals import make_wiki_data_loader

wiki = make_wiki_data_loader(unclustered_model.tokenizer, batch_size=8)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


36718


Map (num_proc=10):   0%|          | 0/36718 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (3325 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3471 > 1024). Running this sequence through the model will result in indexing errors
Map (num_proc=10):   3%|▎         | 1000/36718 [00:00<00:15, 2234.28 examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (3546 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3670 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3652 > 1024). Runn

In [15]:
from transformers import GPT2Model, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

for idx, batch in enumerate(wiki.dataset['tokens']):
    
    batch = batch.to(device)
    print(batch)
    print(tokenizer.decode(batch))
    unclustered_logits = unclustered_model.forward(batch)
    clustered_logits = clustered_model.forward(batch)
    unclustered_predictions = torch.argmax(unclustered_model.forward(batch), dim=-1)
    clustered_predictions = torch.argmax(clustered_model.forward(batch), dim=-1)

    # accuracy on second last token
    unclustered_accuracy = (unclustered_predictions[0, :-1] == batch[1:]).float().mean()
    clustered_accuracy = (clustered_predictions[0, :-1] == batch[1:]).float().mean()
    
    print(tokenizer.decode(batch[-5:]))
    print(tokenizer.decode(clustered_predictions[0, -5:]))

    print(f'Unclustered accuracy: {round(float(100 * unclustered_accuracy), 3)}%')
    print(f'Clustered accuracy: {round(float(100 * clustered_accuracy), 3)}%')

    break

# gt = <end of text> the ...... sbarn 1023
# pred = the ..... sbarn, sdsdfsdf 


/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([50256,   796,   569,  ...,  1398,   764,  2080])
<|endoftext|> = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven ". 
 The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as ma

: 

In [71]:
import nnsight
from nnsight import LanguageModel
# from transformer_lens import HookedTransformer
import torch
import transformers
from transformers import GPT2Model, GPT2Tokenizer
import json
from tqdm import tqdm
import yaml
from matplotlib import pyplot as plt
import seaborn as sns


from transformer_lens.evals import make_wiki_data_loader
from transformer_lens.HookedTransformer import HookedTransformer
import logging
import os
import pickle as pkl
from argparse import ArgumentParser
import numpy as np
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
import transformer_lens
from pprint import pprint
import matplotlib as mpl
from matplotlib.colors import rgb_to_hsv, to_rgb
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model_mod = HookedTransformer.from_pretrained('gpt2-small')
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
device = torch.device("mps")
model_mod.to(device)
model_mod.load_state_dict(torch.load('/Users/maheepchaudhary/pytorch/Projects/nn-modularity/interp-gains-gpt2small/data/models/new_wiki_modular_mlp_in_model_epoch_1.pt', 
                                    map_location=device, 
                                    weights_only=True))


# model_path = "/Users/maheepchaudhary/pytorch/Projects/nn-modularity/interp-gains-gpt2small/data/models/new_wiki_modular_mlp_in_model_epoch_1.pt"

model = AutoModel.from_pretrained("openai-community/gpt2").to(device)
print(model)

gpt2 = LanguageModel(model, tokenizer = tokenizer)
print(gpt2)

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  mps
GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1):

In [72]:
print(model.h[0].attn.c_proj.weight.size())
print(model.h[0].attn.c_attn.weight.size())

torch.Size([768, 768])
torch.Size([768, 2304])


In [73]:
for name, module in model.named_modules():
    print(name, module)

 GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (generator): WrapperModule()
)
wte Embedding(50257, 768)
wpe Embedding(1024, 768)
drop Dropout(p=0.1, inplace=False)
h ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2SdpaAttention(
    

In [ ]:
model.wte.weight = model_mod.W_E

for layer in range(12):
    model.h[layer].mlp.c_fc = model_mod.blocks[layer].mlp.W_in
    model.h[layer].mlp.c_proj = model_mod.blocks[layer].mlp.W_out
    
    model_mod_attn = torch.cat((model_mod.blocks[layer].attn.W_Q, 
                                model_mod.blocks[layer].attn.W_K, 
                                model_mod.blocks[layer].attn.W_V),
                                dim=1)
    
    
    model.h[layer].attn.c_attn.weight = torch.nn.Parameter(model_mod_attn.view(768, -1))
    
    model.h[layer].attn.c_proj.weight = model_mod.blocks[layer].attn.W_O

: 

In [50]:
print(model.h[layer].attn.c_attn.weight.size())

torch.Size([768, 2304])


In [32]:
model_mod

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h